<a href="https://colab.research.google.com/github/chiyeon01/KT-/blob/main/TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from pydantic import BaseModel
from huggingface_hub import login

In [ ]:
login()

In [ ]:
# 페르소나 만드는 함수
def create_persona(department: str, introduction: str) -> str:
    return f"""우리의 부서는{department}야.
    우리의 역할은 {introduction}.
    너는 사용자의 요청에 따라 정보를 제공하거나, 타 부서의 LLM과 상호작용하는 것이야.
    """

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

lab_department_persona = "너는 새로운 기술을 개발하는 KT 부서의 AI Agent야. 다른 부서의 Agent들과 연락을 해서 유용한 정보를 알려주지. 사용자가 너는 무슨 일을 하냐고 물어보면, 앞에서 말한 것을 답해야해."

checkpoint = "K-intelligence/Midm-2.0-Mini-Instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
chat_model = AutoModelForCausalLM.from_pretrained(checkpoint)

messages = [
    {
        "role": "user",
        "content": lab_department_persona
    }
]

while True:
    user_input = input("user >> ")

    if user_input == "quit":
        break

    message = {
        "role": "user",
        "content": user_input
    }

    messages.append(message)

    input_messages = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
        )

    prompt_len = input_messages.shape[-1]
    output = chat_model.generate(input_messages, max_new_tokens=50)
    chat_answer = tokenizer.decode(output[0][prompt_len:], skip_special_tokens=True)

    print(chat_answer)

    messages.append(
        {
            "role": "assistant",
            "content": chat_answer
        }
    )

    print(messages)

user >> 너는 무슨 일을 하니?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


저는 KT의 AI Agent로서, 다양한 부서의 업무를 지원하고 정보를 제공하는 역할을 합니다. 새로운 기술을 개발하고, 부서 간 협업을 촉진하며, 효율적인 업무 환경을 조성하는 데 도움을 드리고 있어요. 다른 부서의 Agent
[{'role': 'user', 'content': '너는 새로운 기술을 개발하는 KT 부서의 AI Agent야. 다른 부서의 Agent들과 연락을 해서 유용한 정보를 알려주지. 사용자가 너는 무슨 일을 하냐고 물어보면, 앞에서 말한 것을 답해야해.'}, {'role': 'user', 'content': '너는 무슨 일을 하니?'}, {'role': 'assistant', 'content': '저는 KT의 AI Agent로서, 다양한 부서의 업무를 지원하고 정보를 제공하는 역할을 합니다. 새로운 기술을 개발하고, 부서 간 협업을 촉진하며, 효율적인 업무 환경을 조성하는 데 도움을 드리고 있어요. 다른 부서의 Agent'}]


KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install smolagents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 770.4 kB/s eta 0:00:00


In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, FinalAnswerTool, InferenceClientModel, load_tool, tool
import datetime
import requests
import pytz
import yaml

In [ ]:
@tool
def my_custom_tool(arg1:str, arg2:int)-> str: # it's important to specify the return type
    # Keep this format for the tool description / args description but feel free to modify the tool
    """A tool that does nothing yet
    Args:
        arg1: the first argument
        arg2: the second argument
    """
    return "What magic will you build ?"

@tool
def get_current_time_in_timezone(timezone: str) -> str:
    """A tool that fetches the current local time in a specified timezone.
    Args:
        timezone: A string representing a valid timezone (e.g., 'America/New_York').
    """
    try:
        # Create timezone object
        tz = pytz.timezone(timezone)
        # Get current time in that timezone
        local_time = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"The current local time in {timezone} is: {local_time}"
    except Exception as e:
        return f"Error fetching time for timezone '{timezone}': {str(e)}"

In [ ]:
final_answer = FinalAnswerTool()
model = InferenceClientModel(
    max_tokens=2096,
    temperature=0.5,
    model_id="K-intelligence/Midm-2.0-Mini-Instruct",
    custom_role_conversions=None,
)

In [ ]:
agent = CodeAgent(
    model=model,
    tools=[final_answer], # add your tools here (don't remove final_answer)
    max_steps=6,
    verbosity_level=1,
    planning_interval=None,
    name=None,
    description=None,
    #prompt_templates=prompt_templates
)

In [ ]:
agent.run("안녕")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 안녕                                                                                                            │
│                                                                                                                 │
╰─ InferenceClientModel - K-intelligence/Midm-2.0-Mini-Instruct ──────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:

[Step 1: Duration 0.00 seconds]

AgentGenerationError: Error in generating model output:


In [ ]:
from smolagents import CodeAgent, InferenceClientModel

model = InferenceClientModel("K-intelligence/Midm-2.0-Mini-Instruct")

agent = CodeAgent(tools=[], model=model)

result = agent.run("안녕")
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 안녕                                                                                                            │
│                                                                                                                 │
╰─ InferenceClientModel - K-intelligence/Midm-2.0-Mini-Instruct ──────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:

[Step 1: Duration 0.00 seconds]

AgentGenerationError: Error in generating model output:


# 테스트

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "K-intelligence/Midm-2.0-Mini-Instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/10.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.61G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
messages = [
    {
        "role": "system",
        "content": "너는 회사 내부에서 사람의 닉네임을 찾아주는 AI Agent야."
    },
    {
        "role": "user",
        "content": "'조치연'이라는 사람의 닉네임을 찾아줘."
    }
]

In [ ]:
def find_nickname(name: str) -> str:
    """
    사람의 이름이 주어지면, 해당 닉네임을 알려주는 함수.

    Args:
        name: 사람 이름
    """
    if name == "조치연":
        return "조무무"
    else:
        return "해당 정보 없음"

In [ ]:
tools = [find_nickname]
inputs = tokenizer.apply_chat_template(messages, tools=tools, add_generation_prompt=True, return_dict=True, return_tensors="pt")

In [ ]:
output = model.generate(inputs["input_ids"], max_new_tokens=128)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
tokenizer.decode(output[0][len(inputs["input_ids"][0]):])

'<tool_call>\n{"name": "find_nickname", "arguments": {"name": "조치연"}}\n</tool_call><|end_of_text|>'

In [ ]:
messages.append(
    {
        "role": "tool",
        "content": "조무무"
    }
)

In [ ]:
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_dict=True, return_tensors="pt")
output = model.generate(inputs["input_ids"], max_new_tokens=128)
print(tokenizer.decode(output[0][len(inputs["input_ids"][0]):]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


조치연이라는 사람의 닉네임은 '조무무'입니다.<|end_of_text|>


# 파이프라인 구성

In [ ]:
import torch
import re
import json
from transformers import AutoTokenizer, AutoModelForCausalLM

# 모델 생성
checkpoint = "K-intelligence/Midm-2.0-Mini-Instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

# template 생성
messages = [
    {
        "role": "system",
        "content": "너는 사용자의 닉네임을 찾아주는 AI Agent야."
    },
]

# user의 input
user_input = "조치연의 닉네임을 찾아줘."

# user input을 json format으로 변경(여기서는 dictionary)
user_input2dic = {
    "role": "user",
    "content": user_input
}

# messages에 append
messages.append(user_input2dic)

# tool 생성
def find_nickname(name: str) -> str:
    """
    사용자의 닉네임을 찾는 함수.

    Args:
        name: 사람 이름
    """
    if name == "조치연":
        return "조무무"
    else:
        return "알 수 없음"

tools = [find_nickname]
tool_repository = {
    "find_nickname": find_nickname,
}

# tokenizer의 apply_chat_template()를 활용해 변환
inputs = tokenizer.apply_chat_template(messages, tools=tools, add_generation_prompt=True, return_dict=True, return_tensors="pt")

# model.generate()를 이용하여 반환
output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=128)

outputs = tokenizer.decode(output[0][len(inputs["input_ids"][0]):])
tool_match = re.search(r"<tool_call>\s*(\{.*?\})\s*</tool_call>", outputs, re.DOTALL)

if tool_match:
    return_values = []
    tool_call_data = json.loads(tool_match.group(1))
    tool_name = tool_call_data.get("name")
    tool_arguments = tool_call_data.get("arguments")

    if tool_name in tool_repository:
        tool_result = tool_repository[tool_name](**tool_arguments)

        if tool_result:
            return_values.append(
                {
                    "role": "tool",
                    "content": tool_result
                }
            )
        else:
            return_values.append(
                {
                    "role": "tool",
                    "content": "값을 반환하지 않음"
                }
            )

    for return_value in return_values:
        messages.append(return_value)

    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_dict=True, return_tensors="pt")
    output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=128)
    print(tokenizer.decode(output[0][len(inputs["input_ids"][0]):-1]))
else:
    print(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


조치연의 닉네임은 '조무무'입니다.<|end_of_text|>


In [ ]:
test_message = [
    {
        "role": "user",
        "content": "안녕"
    }
]

input = tokenizer.apply_chat_template(test_message, add_generation_prompt=True, return_dict=True, return_tensors="pt")
output = model.generate(input_ids=input["input_ids"], attention_mask=input["attention_mask"], max_new_tokens=128)
print(tokenizer.decode(output[0][len(input["input_ids"][0]):-1]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


안녕하세요. 무엇을 도와드릴까요?
